# Analyze the fit results from `sncosmo`

Authors: Midori Rollins and Melissa Graham

*Description to go here.*

## Set up and read in the data

Import packages.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

Read in the `sncosmo` results.

In [2]:
fnm = 'all_fit_snia_files/results_sncosmos.dat'

fit_cand = np.loadtxt(fnm, dtype='str', usecols=(0))
fit_est_z = np.loadtxt(fnm, dtype='float', usecols=(1))
fit_in_zlo = np.loadtxt(fnm, dtype='float', usecols=(2))
fit_in_zhi = np.loadtxt(fnm, dtype='float', usecols=(3))
fit_z = np.loadtxt(fnm, dtype='float', usecols=(4))
fit_ze = np.loadtxt(fnm, dtype='float', usecols=(5))
fit_t0 = np.loadtxt(fnm, dtype='float', usecols=(6))
fit_t0e = np.loadtxt(fnm, dtype='float', usecols=(7))
fit_x0 = np.loadtxt(fnm, dtype='float', usecols=(8))
fit_x0e = np.loadtxt(fnm, dtype='float', usecols=(9))
fit_x1 = np.loadtxt(fnm, dtype='float', usecols=(10))
fit_x1e = np.loadtxt(fnm, dtype='float', usecols=(11))
fit_c = np.loadtxt(fnm, dtype='float', usecols=(12))
fit_ce = np.loadtxt(fnm, dtype='float', usecols=(13))

fit_pmB = np.loadtxt(fnm, dtype='float', usecols=(14))
fit_pmg = np.loadtxt(fnm, dtype='float', usecols=(15))
fit_pmr = np.loadtxt(fnm, dtype='float', usecols=(16))
fit_pmi = np.loadtxt(fnm, dtype='float', usecols=(17))

del fnm

And the review flags.

In [ ]:
fnm = 'all_fit_snia_files/results_sncosmos_fit_review.dat'
fit_flag = np.loadtxt(fnm, dtype='int', usecols=(1))
del fnm

## Convert fit parameters to standard SNIa paramters

We have used `sncosmo` with the SALT-2 lightcurve fitter,
(<a href="https://ui.adsabs.harvard.edu/abs/2007A%26A...466...11G/abstract">Guy et al. 2007</a>).

The five fit parameters are:
 * `z`  : redshift
 * `t0` : time of peak brightness (MJD)
 * `x0` : flux normalization (a proxy for peak brightness)
 * `x1` : a proxy for lightcurve decline rate (or stretch factor)
 * `c`  : color offset
 
### `z` and `t0`

These two are standard, self-explanatory parameters and do not need any further conversions.

### `x0` 

This is the SED normalization in the SALT2 fits, and a conversion from it to 
peak apparent brightness can be done with 
$m_{B} = −2.5\log(x_{0}) + 10.635$ 
from <a href="https://arxiv.org/abs/1401.4065">Mosher et al. (2014)</a>.

However, this isn't necessary as the peak brightness in Bessel B, and sdss g, r, and i 
are all in the fit results file already.


### `x1`

#### To decline rate

$\Delta m_{15}$ is the change in magnitude (it is standard to report this in the B-band)
during the first 15 days after peak brightness, and is commonly refered to as the
decline rate.

Section 5 of Guy+07 gives the conversion between $x_1$ and $\Delta m_{15}$ as:

> $\Delta m_{15} = 1.09 - 0.161x_{1} + 0.013x_{1}^{2} - 0.00130x_{1}^{3}$

SNIa with a larger value of $\Delta m_{15}$ decline faster, and typically have reached
fainter peak brightness.

#### To stretch

The stretch parameter, $s$, is another parameterization of the width or decline rate
of the SNIa which also incorporates the rise time, and was defined in
<a href="https://ui.adsabs.harvard.edu/abs/1997ApJ...483..565P/abstract">Perlmutter et al. 1997</a>.

Section 5 of Guy+07 gives the conversion between $x_1$ and $s$ as:

> $s = 0.98 + 0.091 x_1 + 0.003 x_1^2 - 0.00075 x_1^3$

SNIa with a larger stretch value have a narrower-width lightcurve that has reached a fainter peak.


### `c` 

Section 2 of Guy+07 describes this as 
*"...a color offset with respect to the average at the date maximum luminosity in B-band, 
$c = (B-V)_{\rm MAX} - \left< B-V \right>$ ..."*.

In this case, $(B-V)_{\rm MAX}$ is the color of the fit SNIa at maximum, 
and $\left< B-V \right>$ is the average color for SNIa in general.

What does the value of $c$ indicate about the fit SNIa's color? Let's reason it out:
 * When $c < 0$, that means $(B-V)_{\rm MAX} < \left< B-V \right>$.
 * The fit SNIa has a $B-V$ value that is lower than average.
 * This means either $B$ is smaller or $V$ is larger.
 * Since we're in magnitudes, that means $B$ is brighter or $V$ is fainter.
 * And $B$ being brighter or $V$ being fainter means the SNIa is *bluer*.
 * So when $c < 0$, the fit SNIa is bluer than a typical SNIa.
 * And vice versa, when $c > 0$ the fit SNIa is redder than a typical SNIa.

This is important to understand because SNIa that are intrinsically brighter are bluer,
and we should see this correlation in our fit results.

<br>

<br>

*Next to do: the conversions.*

In [ ]:
import uncertainties as unc  
import uncertainties.unumpy as unp

x1_with_unc = unp.uarray((new_fit_x1, new_fit_x1e))

In [ ]:
delta_m15b=[]
for i in range(len(new_fit_x1)):
    delta_m15b.append(1.09 - (0.161*(x1_with_unc[i])) + (0.013*(x1_with_unc[i])**2) - (0.00130*(x1_with_unc[i])**3))
delta_m15b_e = unp.std_devs(delta_m15b)
delta_m15b_n = unp.nominal_values(delta_m15b)

In [ ]:
# Look at relationship between x1 and delta m15
# plt.scatter(new_fit_x1,delta_m15_n)
plt.errorbar(new_fit_x1,delta_m15b_n,xerr=new_fit_x1e,yerr=delta_m15b_e, fmt="o", elinewidth=1.1, ecolor="red")
plt.xlabel("x1")
plt.ylabel("$\Delta m15$");

In [ ]:
x0_with_unc = unp.uarray((new_fit_x0, new_fit_x0e))

In [ ]:
mb=[]
for i in range(len(new_fit_x0)):
    mb.append((-2.5*unp.log10(x0_with_unc[i]))+10.635)
mb_e = unp.std_devs(mb)
mb_n = unp.nominal_values(mb)

In [ ]:
# Look at relationship between x0 and mb
#plt.scatter(new_fit_x0,mb_n)
plt.errorbar(new_fit_x0,mb_n,xerr=new_fit_x0e,yerr=mb_e, fmt="o", elinewidth=1.1, ecolor="red")
plt.xlabel("x0")
plt.ylabel("mb");

### Calculating $ M_{B}$

In [ ]:
new_fit_z_min = new_fit_z - new_fit_ze
new_fit_z_max = new_fit_z + new_fit_ze

new_fit_x1_min = new_fit_x1 - new_fit_x1e
new_fit_x1_max = new_fit_x1 + new_fit_x1e

new_fit_c_min = new_fit_c - new_fit_ce
new_fit_c_max = new_fit_c + new_fit_ce

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
alpha_min = 0.13-0.013
alpha_max = 0.13+0.013
beta_min = 1.77-0.16
beta_max = 1.77+0.16

abs_mag_min=[]
abs_mag_max=[]
for i in range(len(new_fit_x0)):
    lum_dist_min = cosmo.luminosity_distance(new_fit_z_min[i])
    lum_dist_max = cosmo.luminosity_distance(new_fit_z_max[i])
    dist_mod_min = (5*np.log10(lum_dist_min.value))+25
    dist_mod_max = (5*np.log10(lum_dist_max.value))+25
    abs_mag_max.append((mb_n[i]-dist_mod_min)-(alpha_min*new_fit_x1_min[i])+(beta_min*new_fit_c_min[i]))
    abs_mag_min.append((mb_n[i]-dist_mod_max)-(alpha_max*new_fit_x1_max[i])+(beta_max*new_fit_c_max[i]))
abs_mage = (abs(np.array(abs_mag_min)-np.array(abs_mag_max)))/2
abs_mag = abs_mag_min+abs_mage # or abs_mag_max-abs_mage

In [ ]:
plt.errorbar(delta_m15b_n,abs_mag, xerr=delta_m15b_e, yerr=abs_mage, fmt="o", elinewidth=1.1, ecolor="red")
plt.xlabel("$\Delta m_{15B}$")
plt.ylabel("$M_{B}$")
plt.gca().invert_yaxis();

In [ ]:
plt.errorbar(new_fit_z, mb_n, xerr=new_fit_ze, yerr=mb_e, fmt="o",elinewidth=1.1, ecolor="red")
plt.xlabel('z')
plt.ylabel('mb');

## Comparing relations between parameters

### Comparison between original parameters

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(13, 5)

ax[0].scatter(new_fit_x0, new_fit_x1, color='slateblue')
ax[0].set_xlabel('x0')
ax[0].set_ylabel('x1')

ax[1].scatter(new_fit_x0, new_fit_c, color='orchid')
ax[1].set_xlabel('x0')
ax[1].set_ylabel('c')

ax[2].scatter(new_fit_x1, new_fit_c, color='seagreen')
ax[2].set_xlabel('x1')
ax[2].set_ylabel('c')

fig.tight_layout()
fig.show()

### Comparison between converted parameters

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(13, 5)

ax[0].scatter(mb_n, delta_m15b_n, color='slateblue')
ax[0].set_xlabel("$m_{B}$")
ax[0].set_ylabel("$\Delta m_{15B}$")

ax[1].scatter(mb_n, new_fit_c, color='orchid')
ax[1].set_xlabel("$m_{B}$")
ax[1].set_ylabel('c')

ax[2].scatter(delta_m15b_n, new_fit_c, color='seagreen')
ax[2].set_xlabel("$\Delta m_{15B}$")
ax[2].set_ylabel('c')

fig.tight_layout()
fig.show()

### Comparison between abs mag $M_{B}$, and original & converted parameters

In [ ]:
fig, ax = plt.subplots(2, 3)
fig.set_size_inches(13, 9)

ax[0,0].scatter(mb_n, abs_mag, color='slateblue')
ax[0,0].set_xlabel("$m_{B}$")
ax[0,0].set_ylabel("$M_{B}$")
ax[0,0].invert_yaxis()
ax[0,0].invert_xaxis()

ax[0,1].scatter(delta_m15b_n, abs_mag, color='orchid')
ax[0,1].set_xlabel("$\Delta m_{15B}$")
ax[0,1].set_ylabel("$M_{B}$")
ax[0,1].invert_yaxis()

ax[0,2].scatter(new_fit_z, abs_mag, color='seagreen')
ax[0,2].set_xlabel('z')
ax[0,2].set_ylabel("$M_{B}$")
ax[0,2].invert_yaxis()

ax[1,0].scatter(new_fit_x0, abs_mag, color='maroon')
ax[1,0].set_xlabel('x0')
ax[1,0].set_ylabel("$M_{B}$")
ax[1,0].invert_yaxis()

ax[1,1].scatter(new_fit_x1, abs_mag, color='goldenrod')
ax[1,1].set_xlabel('x1')
ax[1,1].set_ylabel("$M_{B}$")
ax[1,1].invert_yaxis()

ax[1,2].scatter(new_fit_c, abs_mag, color='blueviolet')
ax[1,2].set_xlabel('c')
ax[1,2].set_ylabel("$M_{B}$")
ax[1,2].invert_yaxis()

fig.tight_layout()
fig.show();

### Rating the fits

After looking through the PDF of fits for all candidates, I have given them a rating of good, maybe, or bad.

>**Good**: sncosmo fits our data very well; our data captures both the rise and fall of the supernova; has typical parameter values w/out large errors

>**Maybe**: sncosmo fits our data somewhat well; not enough data/our data only captures the incline/decline of the supernova; may have abnormal parameter values & large errors; missing in a band

>**Bad**: sncosmo does not fit our data well; our data is bad/clearly not type 1a-like; has abnormal parameter values w/ large errors

In [ ]:
good=[1,4,6,7,9,11,12,13,14,19,20,23,24,25,28,37,45,47,59,66,69,70,75,81]
maybe=[3,15,16,17,22,26,27,29,34,35,39,46,51,54,55,56,57,58,60,61,62,63,64,67,68,72,73,79,83,84]
bad=[0,2,5,8,10,18,21,30,31,32,33,36,38,40,41,42,43,44,48,49,50,52,53,65,71,74,76,77,78,80,82]

In [ ]:
# creating flags for good, maybe, and bad indices
flag = np.zeros(len(fit_x1),dtype='int')
flag[good]=0
flag[maybe]=1
flag[bad]=2

# defining lists for our plot parameters
colors=['green','gold','red']
labels=['good','maybe','bad']
names=["x0","x0 error","x1","x1 error","c","c error","z","z error","t0","t0 error"]

# defining various bin values for better visuals
bin1=[2,2,20]
bin2=[5,5,20]
bin3=[10,10,20]
bin4=[15,15,20]

### Plotting the rated distributions

In [ ]:
fig, ax = plt.subplots(5, 2)### Looking at the fit parameter distributions
fig.set_size_inches(16, 16)
plt.subplots_adjust(hspace=0.3)

ax[0,0].hist(fit_x0, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0,0].hist(fit_x0[tx], bins=bin3[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[0,0].set_xlabel("x0")
    ax[0,0].legend()
    
ax[0,1].hist(fit_x0e, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0,1].hist(fit_x0e[tx], bins=bin1[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[0,1].set_xlabel("x0 error")
    # ax[0,1].legend()

ax[1,0].hist(fit_x1, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1,0].hist(fit_x1[tx], bins=bin2[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[1,0].set_xlabel("x1")
    # ax[1,0].legend()
    
ax[1,1].hist(fit_x1e, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1,1].hist(fit_x1e[tx], bins=bin1[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[1,1].set_xlabel("x1 error")
    # ax[1,1].legend()
    
ax[2,0].hist(fit_c, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[2,0].hist(fit_c[tx], bins=bin2[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[2,0].set_xlabel("c")
    # ax[2,0].legend()
    
ax[2,1].hist(fit_ce, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[2,1].hist(fit_ce[tx], bins=bin2[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[2,1].set_xlabel("c error")
    # ax[2,1].legend()
    
ax[3,0].hist(fit_z, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[3,0].hist(fit_z[tx], bins=bin4[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[3,0].set_xlabel("z")
    # ax[3,0].legend()
    
ax[3,1].hist(fit_ze, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[3,1].hist(fit_ze[tx], bins=bin3[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[3,1].set_xlabel("z error")
    # ax[3,1].legend()
    
ax[4,0].hist(fit_t0, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[4,0].hist(fit_t0[tx], bins=bin4[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[4,0].set_xlabel("t0")
    # ax[4,0].legend()
    
ax[4,1].hist(fit_t0e, bins=20, density=False, histtype='stepfilled', color='lightgrey', label='all values')
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[4,1].hist(fit_t0e[tx], bins=bin1[i], density=False, histtype='step', color=colors[i], label=labels[i])
    ax[4,1].set_xlabel("t0 error")
    # ax[4,1].legend()

### Recomparing relations between parameters w/ rated data

#### Comparison between original parameters w/ rated data

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(15, 5)
plt.subplots_adjust(wspace=0.3)

for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0].scatter(fit_x0[tx], fit_x1[tx],color=colors[i], label=labels[i])
    ax[0].set_xlabel("x0")
    ax[0].set_ylabel("x1")
    ax[0].legend()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1].scatter(fit_x0[tx], fit_c[tx], color=colors[i], label=labels[i])
    ax[1].set_xlabel("x0")
    ax[1].set_ylabel("c")
    ax[1].legend()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[2].scatter(fit_x1[tx], fit_c[tx],color=colors[i], label=labels[i])
    ax[2].set_xlabel("x1")
    ax[2].set_ylabel("c")
    ax[2].legend()


#### Comparison between converted parameters w/ rated data

In [ ]:
x1_with_unc = unp.uarray((fit_x1, fit_x1e))
delta_m15b=[]
for i in range(len(fit_x1)):
    delta_m15b.append(1.09 - (0.161*(x1_with_unc[i])) + (0.013*(x1_with_unc[i])**2) - (0.00130*(x1_with_unc[i])**3))
delta_m15b_e = unp.std_devs(delta_m15b)
delta_m15b_n = unp.nominal_values(delta_m15b)

In [ ]:
x0_with_unc = unp.uarray((fit_x0, fit_x0e))
mb=[]
for i in range(len(fit_x0)):
    mb.append((-2.5*unp.log10(x0_with_unc[i]))+10.635)
mb_e = unp.std_devs(mb)
mb_n = unp.nominal_values(mb)

In [ ]:
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

fit_z_min = fit_z - fit_ze
fit_z_max = fit_z + fit_ze
fit_x1_min = fit_x1 - fit_x1e
fit_x1_max = fit_x1 + fit_x1e
fit_c_min = fit_c - fit_ce
fit_c_max = fit_c + fit_ce

abs_mag_min=[]
abs_mag_max=[]
for i in range(len(fit_x0)):
    lum_dist_min = cosmo.luminosity_distance(fit_z_min[i])
    lum_dist_max = cosmo.luminosity_distance(fit_z_max[i])
    dist_mod_min = (5*np.log10(lum_dist_min.value))+25
    dist_mod_max = (5*np.log10(lum_dist_max.value))+25
    abs_mag_max.append((mb_n[i]-dist_mod_min)-(alpha_min*fit_x1_min[i])+(beta_min*fit_c_min[i]))
    abs_mag_min.append((mb_n[i]-dist_mod_max)-(alpha_max*fit_x1_max[i])+(beta_max*fit_c_max[i]))
abs_mage = (abs(np.array(abs_mag_min)-np.array(abs_mag_max)))/2
abs_mag = abs_mag_min+abs_mage # or abs_mag_max-abs_mage

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(12, 5)
plt.subplots_adjust(wspace=0.3)

for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0].scatter(mb_n[tx], delta_m15b_n[tx], color=colors[i], label=labels[i])
    ax[0].set_xlabel("$m_{B}$")
    ax[0].set_ylabel("$\Delta m_{15B}$")
    ax[0].legend()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1].scatter(mb_n[tx], fit_c[tx], color=colors[i], label=labels[i])
    ax[1].set_xlabel("$m_{B}$")
    ax[1].set_ylabel('c')
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[2].scatter(delta_m15b_n[tx], fit_c[tx],color=colors[i], label=labels[i])
    ax[2].set_xlabel("$\Delta m_{15B}$")
    ax[2].set_ylabel('c')


#### Comparison between abs mag $M_{B}$, and original & converted parameters

In [ ]:
fig, ax = plt.subplots(2, 3)
fig.set_size_inches(15, 11)

for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0,0].scatter(mb_n[tx], abs_mag[tx], color=colors[i])
    ax[0,0].set_xlabel("$m_{B}$")
    ax[0,0].set_ylabel("$M_{B}$")
    ax[0,0].invert_yaxis()
    ax[0,0].invert_xaxis()

for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0,1].scatter(delta_m15b_n[tx], abs_mag[tx], color=colors[i])
    ax[0,1].set_xlabel("$\Delta m_{15B}$")
    ax[0,1].set_ylabel("$M_{B}$")
    ax[0,1].invert_yaxis()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[0,2].scatter(fit_z[tx], abs_mag[tx], color=colors[i])
    ax[0,2].set_xlabel('z')
    ax[0,2].set_ylabel("$M_{B}$")
    ax[0,2].invert_yaxis()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1,0].scatter(fit_x0[tx], abs_mag[tx], color=colors[i])
    ax[1,0].set_xlabel('x0')
    ax[1,0].set_ylabel("$M_{B}$")
    ax[1,0].invert_yaxis()
    
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1,1].scatter(fit_x1[tx], abs_mag[tx], color=colors[i])
    ax[1,1].set_xlabel('x1')
    ax[1,1].set_ylabel("$M_{B}$")
    ax[1,1].invert_yaxis()
    
for i in range(3):
    tx=np.where(flag==i)[0]
    ax[1,2].scatter(fit_c[tx], abs_mag[tx], color=colors[i])
    ax[1,2].set_xlabel('c')
    ax[1,2].set_ylabel("$M_{B}$")
    ax[1,2].invert_yaxis()

<!-- ### Other Analysis
> **True Good:** Candidates that had normal parameter/error values AND had good fits
> **True Bad:** Candidates that had outlying parameters/large errors AND had bad fits
> **False_Good:** Candidates that had normal parameter/error values AND had bad fits
> **False_Bad:** Candidates that had outlying parameters/large errors AND had good fits -->